Using kernel `conda_pytorch_latest_p36`

In [19]:
# !pip install transformers
# !pip install datasets

In [6]:
import sys
sys.path.append('../../../')

In [4]:
from pathlib import Path
import os
import random

In [5]:
import pandas as pd
import numpy as np
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TrainingArguments
from sklearn.metrics import classification_report
import torch

In [7]:
from deep.constants import *

In [11]:
data = IMMAP_PATH / 'final_data/en/'

In [12]:
train = pd.read_csv(data / 'sentences_en_train.csv')
test = pd.read_csv(data / 'sentences_en_test.csv')

In [42]:
train

doc_id  sentence_id  \
0        51787           38   
1        51787           44   
2        51787           45   
3        51787           49   
4        51787           53   
...        ...          ...   
191932   34512          112   
191933   34512          116   
191934   34512          120   
191935   34512          124   
191936   34512          125   

                                            sentence_text  is_relevant  \
0       Climate Change and Agriculture: Subsistence Fa...            0   
1        Bohorquez-Penuela, C., & Otero-Cortes, A (2020).            0   
2       Blame it on the Rain: The Effects of Weather S...            0   
3       Increasing frequency of extreme El Ni o events...            0   
4                      What do we learn from the weather?            0   
...                                                   ...          ...   
191932  Emergency shelters in India had reportedly bee...            0   
191933  In these situations humanitarian operations ma...            0   
191934  Governments of EU member states and several Af...            0   
191935  You can find an overview of all ACAPS resource...            0   
191936  11 ACAPS HUMANITARIAN ACCESS OVERVIEW JULY 202...            0   

       sector_ids  
0              []  
1              []  
2              []  
3              []  
4              []  
...           ...  
191932         []  
191933         []  
191934         []  
191935         []  
191936         []  

[191937 rows x 5 columns]

In [50]:
from datasets import list_datasets, load_dataset, list_metrics, load_metric

# Print all the available datasets
a = load_dataset('csv', data_files=data / 'sentences_en_train.csv')

ValueError: Please provide a valid `data_files` in `DatasetBuilder`

In [46]:
a['train']['text']

['Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!',
 'Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything fro

In [29]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

In [13]:
train_encodings = tokenizer(list(train_df.sentence_text), truncation=True, padding=True)
train_labels = list(train_df.sector_ids)

In [14]:
test_encodings = tokenizer(list(test_df.sentence_text), truncation=True, padding=True)
test_labels = list(test_df.sector_ids)

In [15]:
import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_labels)
test_dataset = Dataset(test_encodings, test_labels)

In [16]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=train_dataset            # evaluation dataset
)

In [17]:
len(train_dataset)

22972

In [18]:
torch.cuda.is_available()

True

In [ ]:
trainer.train()

In [ ]:
outputs = trainer.predict(test_dataset)

In [ ]:
preds = np.argmax(outputs.predictions, axis=1)
labels = outputs.label_ids

In [ ]:
rep = classification_report(labels, preds)

In [24]:
print(rep)

              precision    recall  f1-score   support

           0       0.86      0.93      0.89      1646
           1       0.90      0.80      0.85      1260

    accuracy                           0.88      2906
   macro avg       0.88      0.87      0.87      2906
weighted avg       0.88      0.88      0.87      2906

